# NFL Weather Analysis
## Project 1 - Group3

In [1]:
# import dependencies
import pandas as pd


### Data Import
The stadium, team, and gameday location data are gathered from the "NFL scores and betting data" Kaggle dataset:
https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data

In [108]:
def fn_ImportDataToFrames(arg_dictDataFiles, arg_strEnconding = "UTF-8"):
    '''
    FUNCTION: fn_ImportDataForWeatherAnalysis
    
    DESCRIPTION: Reads csv data from dictionary and populated into dictionary of dataframes
    
    ARGUMENTS:
        arg_dictDataFiles:  Dictionary containing strings pointing to csv input files
        arg_strEncoding:    String specifying encoding used in files
        
    RETURNS:
        Dictionary matching length and keys of arg_dictDataFiles but with populated dataframes
        instead of filename strings
    '''    
    
    # define dictionary to be returned
    local_dictDataFrames = {}
    
    # read each of the files from the dictionary arg into a dataframe
    for key in arg_dictDataFiles.keys():
        local_dictDataFrames[key] = pd.read_csv(arg_dictDataFiles[key], encoding = arg_strEnconding)

    return local_dictDataFrames

# define filename strings where csv data resides
dict_DataFiles = {
    "Stadiums": "../data/nfl_stadiums.csv",
    "Teams": "../data/nfl_teams.csv",
    "Games": "../data/spreadspoke_scores.csv"
}

# populate dataframes with csv data from files
dict_DataFrames = fn_ImportDataToFrames(dict_DataFiles, "ISO-8859-1")

### Reduce Datasets to Relevant Data

In [109]:
def fn_ReduceData(arg_dictDataFrames, arg_dictColsToDrop, arg_dictRowCriteria):
    '''
    FUNCTION: fn_ReduceData
    
    DESCRIPTION: Reduces dataframes by dropping specified columns and only keeping specified rows
    
    ARGUMENTS:
        arg_dictDataFrames: Dictionary containing dataframes to be reduced
        arg_dictColsToDrop: Dictionary containing lists of columns to be dropped for each dataframe
        arg_dictRowCriteria: Dictionary containing lists of booleans indicating which rows to keep
        
    RETURNS:
        Dictionary matching length of arg_dictDataFrames but with reduced shape
    '''    
    
    # define dictionary to be returned
    local_dictDataFrames = {}


    # iterate through dataframes removing columns and filtering rows
    for key in arg_dictDataFrames.keys():
        
        # get cols to remove
        list_cols = arg_dictColsToDrop[key]
        
        # get row filter criteria
        list_criteria = arg_dictRowCriteria[key]
        
        # reduce dataframe columns
        if (len(list_cols) > 0):
            local_dictDataFrames[key] = arg_dictDataFrames[key].drop(list_cols, axis=1)
        
        # reduce dataframe rows
        if (len(list_criteria) > 0):
            local_dictDataFrames[key] = local_dictDataFrames[key][list_criteria]
        
    return local_dictDataFrames

# define lists of columns to be dropped from each dataframe
dict_ColsToDrop = {
    "Stadiums": ["stadium_location", "stadium_open","stadium_close","stadium_address",\
                           "stadium_weather_type","stadium_capacity","stadium_surface","NAME","ELEVATION"],
    "Teams": ["team_name_short","team_id_pfr","team_conference","team_division","team_conference_pre2002",\
                        "team_division_pre2002"],
    "Games": ["schedule_week","team_favorite_id","spread_favorite","over_under_line","stadium_neutral"]
}

# setup boolean criteria to identify rows within our analysis years (2009-2018)
dict_FilterCriteria = {
    "Stadiums": [],
    "Teams": [],
    "Games": (dict_DataFrames["Games"]["schedule_season"]>=2009) & (dict_DataFrames["Games"]["schedule_season"]<=2018)
}

# reduce dataframe columns and rows
dict_DataFrames = fn_ReduceData(dict_DataFrames, dict_ColsToDrop, dict_FilterCriteria)


### Enrich Data

##### Add Team Ids to game data by merging team and game frames on team name


In [110]:
df_Games = dict_DataFrames["Games"]
df_Teams = dict_DataFrames["Teams"]
df_Stadiums = dict_DataFrames["Stadiums"]

# merge on home team first
df_Games = df_Games.merge(df_Teams, how="left", left_on="team_home", right_on="team_name")

# rename new ID column to indicate it is home team
df_Games.rename({"team_id":"team_id_home"},axis=1,inplace=True)

# merge on away team next
df_Games = df_Games.merge(df_Teams, how="left", left_on="team_away", right_on="team_name")

# rename new ID column to indicate it is home team
df_Games.rename({"team_id":"team_id_away"},axis=1,inplace=True)

# cleanup unnecessary columns
df_Games.drop(["team_name_y","team_name_x"],axis=1,inplace=True)

##### Add Weather Station Code, Lat, and Long by merging stadium and game frames on stadium name

In [112]:
# merge on stadium name
df_Games = df_Games.merge(df_Stadiums, how="left", left_on="stadium", right_on="stadium_name")

# cleanup unnecessary columns
df_Games.drop(["stadium_name"],axis=1,inplace=True)

In [113]:
# filter to outdoor stadiums as we are only interested in outdoor games for weather analysis. also kick out NaN STATION rows.
df_Games = df_Games[(df_Games["stadium_type"] == "outdoor") & (pd.notnull(df_Games["STATION"]))]


---

### Get Weather Data For Each Game

In [22]:
# import NOAA api key
from config import noaa_key

# import other dependencies
import requests
import time

# define dataframe
df_weather = pd.DataFrame()

# base url to query the weather api (GHCND dataset for Daily Summary)
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"

# setup api token in header
dict_headers = {"token":noaa_key}

# setup query params for api
#### dataset id for Daily Summary: GHCND
#### datatype id for rainfall: PRCP
#### datatype id for snowfall: SNOW
#### startdate 
#### enddate
#### stationid

str_DataSetId = "GHCND"
list_datatypeids = ["PRCP","SNOW"]

str_UrlParam1 = f"datasetid={str_DataSetId}"

for dataType in list_datatypeids:
    str_UrlParam1 += f"&datatypeid={dataType}"

# define iteration counter. when a multiple of 5, sleep for 1 second to avoid issues with NOAA
int_Counter = 1


for index, row in df_Games.iterrows():
    try:
        # convert date to ISO format
        game_date = row["schedule_date"]
        iso_date_list = game_date.split("/")
        iso_date = iso_date_list[2]+"-"+iso_date_list[0]+"-"+iso_date_list[1]

        str_UrlParam2 = str_UrlParam1 + f"&startdate={iso_date}&enddate={iso_date}"
        str_UrlParam2 += f"&stationid={str_DataSetId}:{row['STATION']}"

        # call the API
        print(f"query url: {base_url + str_UrlParam2}")
        response = requests.get(base_url + str_UrlParam2, headers=dict_headers)
        print(f"API HTML Response: {response}")
        print(response.text)

        # grab amount of rainfall (datatype:PRCP, value:amount in mm) and snowfall (datatype:SNOW, value:amount in mm)
        df_weather = df_weather.append(pd.DataFrame.from_dict(response.json()["results"]))

        int_Counter += 1

        # if counter is a multiple of 5, sleep for 1 seconds
        if ((int_Counter % 5) == 0):
            print("sleeping for 1 secs")
            time.sleep(1)

    except Exception as e:
        print(f"Failed while calling API: {str(e)}")

query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-09-10&enddate=2009-09-10&stationid=GHCND:USW00094823
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2009-09-10T00:00:00","datatype":"PRCP","station":"GHCND:USW00094823","attributes":"T,,0,2400","value":0},{"date":"2009-09-10T00:00:00","datatype":"SNOW","station":"GHCND:USW00094823","attributes":",,0,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-09-13&enddate=2009-09-13&stationid=GHCND:USW00093721
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2009-09-13T00:00:00","datatype":"PRCP","station":"GHCND:USW00093721","attributes":",,0,2400","value":0},{"date":"2009-09-13T00:00:00","datatype":"SNOW","station":"GHCND:USW00093721","attributes":",,0,","val

API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":1,"limit":25}},"results":[{"date":"2009-09-20T00:00:00","datatype":"PRCP","station":"GHCND:USW00093107","attributes":",,W,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-09-20&enddate=2009-09-20&stationid=GHCND:USW00023234
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":1,"limit":25}},"results":[{"date":"2009-09-20T00:00:00","datatype":"PRCP","station":"GHCND:USW00023234","attributes":",,0,2400","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-09-20&enddate=2009-09-20&stationid=GHCND:USW00013897
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2009-09-20T00:00:00","datatype":"PRCP","station":"GHCND:USW00013897","attributes":",,0,2400","valu

API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2009-10-04T00:00:00","datatype":"PRCP","station":"GHCND:USC00190860","attributes":",,0,2400","value":0},{"date":"2009-10-04T00:00:00","datatype":"SNOW","station":"GHCND:USC00190860","attributes":",,0,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-10-04&enddate=2009-10-04&stationid=GHCND:USW00094823
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2009-10-04T00:00:00","datatype":"PRCP","station":"GHCND:USW00094823","attributes":",,0,2400","value":0},{"date":"2009-10-04T00:00:00","datatype":"SNOW","station":"GHCND:USW00094823","attributes":",,0,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-10-04&enddate=2009-10-04&stationid=GHCND:USW0

API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":1,"limit":25}},"results":[{"date":"2009-10-18T00:00:00","datatype":"PRCP","station":"GHCND:USW00023230","attributes":",,W,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-10-18&enddate=2009-10-18&stationid=GHCND:USW00094823
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2009-10-18T00:00:00","datatype":"PRCP","station":"GHCND:USW00094823","attributes":",,0,2400","value":0},{"date":"2009-10-18T00:00:00","datatype":"SNOW","station":"GHCND:USW00094823","attributes":",,0,","value":0}]}
sleeping for 1 secs
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-10-18&enddate=2009-10-18&stationid=GHCND:US1WAKG0038
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":1,"l

API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2009-11-01T00:00:00","datatype":"PRCP","station":"GHCND:USW00013897","attributes":",,0,2400","value":0},{"date":"2009-11-01T00:00:00","datatype":"SNOW","station":"GHCND:USW00013897","attributes":",,0,","value":0}]}
sleeping for 1 secs
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-11-08&enddate=2009-11-08&stationid=GHCND:USC00111550
API HTML Response: <Response [200]>
{}
Failed while calling API: 'results'
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-11-08&enddate=2009-11-08&stationid=GHCND:USW00003871
API HTML Response: <Response [200]>
{}
Failed while calling API: 'results'
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2009-11-08&enddate=2009-11-08&stationi

...

API HTML Response: <Response [200]>
{}
Failed while calling API: 'results'
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-11-11&enddate=2018-11-11&stationid=GHCND:USW00003871
API HTML Response: <Response [200]>
{}
Failed while calling API: 'results'
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-11-11&enddate=2018-11-11&stationid=GHCND:USW00014820
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2018-11-11T00:00:00","datatype":"PRCP","station":"GHCND:USW00014820","attributes":",,W,2400","value":0},{"date":"2018-11-11T00:00:00","datatype":"SNOW","station":"GHCND:USW00014820","attributes":",,W,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-11-11&enddate=2018-11-11&stationid=GHCND:USW00014898


API HTML Response: <Response [200]>
{}
Failed while calling API: 'results'
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-11-25&enddate=2018-11-25&stationid=GHCND:USW00013881
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2018-11-25T00:00:00","datatype":"PRCP","station":"GHCND:USW00013881","attributes":"T,,W,2400","value":0},{"date":"2018-11-25T00:00:00","datatype":"SNOW","station":"GHCND:USW00013881","attributes":",,W,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-11-25&enddate=2018-11-25&stationid=GHCND:USW00003871
API HTML Response: <Response [200]>
{}
Failed while calling API: 'results'
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-11-25&enddate=2018-11-25&stationid=GHCND:USW00023062

API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2018-12-06T00:00:00","datatype":"PRCP","station":"GHCND:USW00013897","attributes":",,W,2400","value":0},{"date":"2018-12-06T00:00:00","datatype":"SNOW","station":"GHCND:USW00013897","attributes":",,W,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-12-09&enddate=2018-12-09&stationid=GHCND:US1NYER0093
API HTML Response: <Response [200]>
{}
Failed while calling API: 'results'
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-12-09&enddate=2018-12-09&stationid=GHCND:USC00111550
API HTML Response: <Response [200]>
{}
Failed while calling API: 'results'
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-12-09&enddate=2018-12-09&stationid=GHCND:USW00014820


query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-12-16&enddate=2018-12-16&stationid=GHCND:USW00014734
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2018-12-16T00:00:00","datatype":"PRCP","station":"GHCND:USW00014734","attributes":",,W,2400","value":373},{"date":"2018-12-16T00:00:00","datatype":"SNOW","station":"GHCND:USW00014734","attributes":",,W,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-12-16&enddate=2018-12-16&stationid=GHCND:USW00094823
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2018-12-16T00:00:00","datatype":"PRCP","station":"GHCND:USW00094823","attributes":",,W,2400","value":132},{"date":"2018-12-16T00:00:00","datatype":"SNOW","station":"GHCND:USW00094823","attributes":",,W,","

API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2018-12-30T00:00:00","datatype":"PRCP","station":"GHCND:USW00023062","attributes":",,7,0600","value":0},{"date":"2018-12-30T00:00:00","datatype":"SNOW","station":"GHCND:USW00023062","attributes":",,7,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-12-30&enddate=2018-12-30&stationid=GHCND:USW00014898
API HTML Response: <Response [200]>
{"metadata":{"resultset":{"offset":1,"count":2,"limit":25}},"results":[{"date":"2018-12-30T00:00:00","datatype":"PRCP","station":"GHCND:USW00014898","attributes":",,W,2400","value":0},{"date":"2018-12-30T00:00:00","datatype":"SNOW","station":"GHCND:USW00014898","attributes":",,W,","value":0}]}
query url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&datatypeid=SNOW&startdate=2018-12-30&enddate=2018-12-30&stationid=GHCND:US1M

In [25]:
df_weather.tail(200)

,attributes,datatype,date,station,value
0,"T,,W,2400",PRCP,2018-10-07T00:00:00,GHCND:USW00014734,0
1,",,W,",SNOW,2018-10-07T00:00:00,GHCND:USW00014734,0
0,"T,,W,2400",PRCP,2018-10-07T00:00:00,GHCND:USW00013739,0
1,",,W,",SNOW,2018-10-07T00:00:00,GHCND:USW00013739,0
0,",,W,2400",PRCP,2018-10-07T00:00:00,GHCND:USW00094823,0
1,",,W,",SNOW,2018-10-07T00:00:00,GHCND:USW00094823,0
0,",,W,2400",PRCP,2018-10-07T00:00:00,GHCND:USW00023234,0
1,",,H,",SNOW,2018-10-07T00:00:00,GHCND:USW00023234,0
0,",,W,2400",PRCP,2018-10-11T00:00:00,GHCND:USW00014734,183
1,",,W,",SNOW,2018-10-11T00:00:00,GHCND:USW00014734,0


In [26]:
# store api results to csv file so we don't have to call API again
df_weather.to_csv("../data/weather.csv", index=False)

##### Prep API data to join back to game data

In [29]:
df_weather["station"] = df_weather["station"].str.split(":")

,attributes,datatype,date,station,value
0,"T,,0,2400",PRCP,2009-09-10T00:00:00,"[GHCND, USW00094823]",0
1,",,0,",SNOW,2009-09-10T00:00:00,"[GHCND, USW00094823]",0
0,",,0,2400",PRCP,2009-09-13T00:00:00,"[GHCND, USW00093721]",0
1,",,0,",SNOW,2009-09-13T00:00:00,"[GHCND, USW00093721]",0
0,",,0,2400",PRCP,2009-09-13T00:00:00,"[GHCND, USW00013881]",0


In [33]:
df_weather["station_name"] = df_weather.station.map(lambda x: x[1])
df_weather.head()

,attributes,datatype,date,station,value,station_name
0,"T,,0,2400",PRCP,2009-09-10T00:00:00,"[GHCND, USW00094823]",0,USW00094823
1,",,0,",SNOW,2009-09-10T00:00:00,"[GHCND, USW00094823]",0,USW00094823
0,",,0,2400",PRCP,2009-09-13T00:00:00,"[GHCND, USW00093721]",0,USW00093721
1,",,0,",SNOW,2009-09-13T00:00:00,"[GHCND, USW00093721]",0,USW00093721
0,",,0,2400",PRCP,2009-09-13T00:00:00,"[GHCND, USW00013881]",0,USW00013881


In [95]:
# pivot PRCP and SNOW rows to columns instead
df_weather_pivot = df_weather.pivot_table(index=["date","station_name"],columns="datatype",values="value")
df_weather_pivot.head()

datatype                          PRCP  SNOW
date                station_name            
2009-09-10T00:00:00 USW00094823    0.0   0.0
2009-09-13T00:00:00 US1WAKG0038    0.0   0.0
                    USW00012842   20.0   0.0
                    USW00013881    0.0   0.0
                    USW00014734    0.0   0.0

In [96]:
# move indexes back to columns
df_weather_pivot = df_weather_pivot.fillna(0).reset_index()

# create boolean to indicate if it rained or snowed on a given date at a given station
df_weather_pivot["rain_or_snow"] = df_weather_pivot["PRCP"].astype(bool) | df_weather_pivot["SNOW"].astype(bool)

In [97]:
df_weather_pivot.head()

datatype,date,station_name,PRCP,SNOW,rain_or_snow
0,2009-09-10T00:00:00,USW00094823,0.0,0.0,False
1,2009-09-13T00:00:00,US1WAKG0038,0.0,0.0,False
2,2009-09-13T00:00:00,USW00012842,20.0,0.0,True
3,2009-09-13T00:00:00,USW00013881,0.0,0.0,False
4,2009-09-13T00:00:00,USW00014734,0.0,0.0,False


In [98]:
# reformat date back to original format
df_weather_pivot["date"] = df_weather_pivot["date"].str[0:10]
df_dates = pd.DataFrame(df_weather_pivot["date"].str.split("-"))

df_dates_split = pd.DataFrame(df_dates["date"].values.tolist())

df_dates_split["schedule_date"] = df_dates_split[1]+"/"+df_dates_split[2]+"/"+df_dates_split[0]
del df_dates_split[0]
del df_dates_split[1]
del df_dates_split[2]

df_dates_split.head()
df_weather_pivot = df_weather_pivot.join(df_dates_split)

df_weather_pivot = df_weather_pivot.rename(columns={"station_name":"STATION"})
df_weather_pivot.head()

,date,STATION,PRCP,SNOW,rain_or_snow,schedule_date
0,2009-09-10,USW00094823,0.0,0.0,False,09/10/2009
1,2009-09-13,US1WAKG0038,0.0,0.0,False,09/13/2009
2,2009-09-13,USW00012842,20.0,0.0,True,09/13/2009
3,2009-09-13,USW00013881,0.0,0.0,False,09/13/2009
4,2009-09-13,USW00014734,0.0,0.0,False,09/13/2009


In [123]:
# merge weather data back to game data
df_merged_weather = df_Games.merge(df_weather_pivot, on=["STATION","schedule_date"])
df_merged_weather = df_merged_weather[["schedule_date","team_id_home","team_id_away","rain_or_snow"]]
df_merged_weather.head(10)

,schedule_date,team_id_home,team_id_away,rain_or_snow
0,09/10/2009,PIT,TEN,False
1,09/13/2009,BAL,KC,False
2,09/13/2009,CAR,PHI,False
3,09/13/2009,CLE,MIN,False
4,09/13/2009,GB,CHI,False
5,09/13/2009,NYG,WAS,False
6,09/13/2009,SEA,LAR,False
7,09/13/2009,TB,DAL,True
8,09/14/2009,NE,BUF,False
9,09/14/2009,OAK,LAC,False


In [124]:
# output weather summary to new file
df_merged_weather.to_csv("../data/weather_summary.csv",index=False)